Intro ideas...
- Asteroids and comets and moving targets, they won't have a fixed RA and Dec. 
- "While its primary mission avoids the vicinity of the ecliptic plane by approximately six degrees, the scale height of the solar system debris disk is large enough to place various small body populations in the field of view." https://ui.adsabs.harvard.edu/abs/2020ApJS..247...26P/abstract <- They have MAST data (TSSYS-DR1) and published lightcurves and periods. We'll get the TESS cutout by object name and compare.
- What is a CUTOUT, what is an FFI TIME SERIES. FFI = full-frame images, as opposed to target pixel files (TPFs) https://heasarc.gsfc.nasa.gov/docs/tess/data-products.html 

In [ ]:
# 

TESScut citation https://ui.adsabs.harvard.edu/abs/2019ascl.soft05007B/abstract Brasseur et al. 2019